<a href="https://colab.research.google.com/github/smilesarah/Python/blob/main/World_dataset_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the world data in JSON format:

In [ ]:
import json
url = "https://progzoo.net/world.json"
from urllib.request import urlopen

raw = urlopen(url).read()
json_data = json.loads(raw)
json_data

{'result': [{'gdp': 20364000000,
   'name': 'Afghanistan',
   'area': 652230,
   'tld': '.af',
   'id': 0,
   'flag': '//upload.wikimedia.org/wikipedia/commons/5/5c/Flag_of_the_Taliban.svg',
   'capital': 'Kabul',
   'continent': 'Asia',
   'population': 25500100},
  {'gdp': 12044000000,
   'name': 'Albania',
   'area': 28748,
   'tld': '.al',
   'id': 1,
   'flag': '//upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg',
   'capital': 'Tirana',
   'continent': 'Europe',
   'population': 2821977},
  {'gdp': 207021000000,
   'name': 'Algeria',
   'area': 2381741,
   'tld': '.dz',
   'id': 2,
   'flag': '//upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Algeria.svg',
   'capital': 'Algiers',
   'continent': 'Africa',
   'population': 38700000},
  {'gdp': 3222000000,
   'name': 'Andorra',
   'area': 468,
   'tld': '.ad',
   'id': 3,
   'flag': '//upload.wikimedia.org/wikipedia/commons/1/19/Flag_of_Andorra.svg',
   'capital': 'Andorra la Vella',
   'continent': 'Europe',
  

# Use the `.DataFrame.from_dict()` method to make the json object into a pandas dataframe.

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(json_data)
df.head()

result
0  {'gdp': 20364000000, 'name': 'Afghanistan', 'a...
1  {'gdp': 12044000000, 'name': 'Albania', 'area'...
2  {'gdp': 207021000000, 'name': 'Algeria', 'area...
3  {'gdp': 3222000000, 'name': 'Andorra', 'area':...
4  {'gdp': 116308000000, 'name': 'Angola', 'area'...

# Using the series of dictionaries above, load the following dataframe below with the columns of `['gdp','name','area','capital','continent','population']`.

In [ ]:
for col in ['gdp','name','area','capital','continent','population']:
  df[col]=df['result'].apply(lambda x: x[col])
df.drop('result', axis=1,inplace=True)
df.head()

gdp         name     area           capital continent  population
0  2.036400e+10  Afghanistan   652230             Kabul      Asia    25500100
1  1.204400e+10      Albania    28748            Tirana    Europe     2821977
2  2.070210e+11      Algeria  2381741           Algiers    Africa    38700000
3  3.222000e+09      Andorra      468  Andorra la Vella    Europe       76098
4  1.163080e+11       Angola  1246700            Luanda    Africa    19183590

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
df.to_csv('/content/drive/MyDrive/world.csv')

Mounted at /content/drive/


# Here we set up the sqlite connection. In this case, it runs locally and don't need to specify a database.

In [ ]:
import sqlite3
conn = sqlite3.connect(':memory:')
df.to_sql('world',conn,if_exists='replace',index=False)

195

# Show a preview of the data frame:

In [ ]:
query = '''
SELECT *
FROM world
'''

result = pd.read_sql_query(query,conn)
result.head()

gdp         name     area           capital continent  population
0  2.036400e+10  Afghanistan   652230             Kabul      Asia    25500100
1  1.204400e+10      Albania    28748            Tirana    Europe     2821977
2  2.070210e+11      Algeria  2381741           Algiers    Africa    38700000
3  3.222000e+09      Andorra      468  Andorra la Vella    Europe       76098
4  1.163080e+11       Angola  1246700            Luanda    Africa    19183590

# Problem 1: For each continent, find the number of countries it contains, the total population, and total area.

In [ ]:
query1 = '''
SELECT continent, COUNT(name), SUM(population) AS total_population, SUM(area) AS total_population
FROM world
GROUP BY continent;
'''
result1 = pd.read_sql_query(query1,conn)
result1

continent  COUNT(name)  total_population  total_population
0         Africa           53        1015844577          29047819
1           Asia           47        4342955676          30156669
2      Caribbean           11          36149204            218887
3        Eurasia            2         149017400          17154985
4         Europe           44         610261850           8714329
5  North America           11         518755156          22298259
6        Oceania           14          37783477           8490477
7  South America           13         407618970          17738064

# Problem 2: Are any countries with more than 1 listed capital?

In [ ]:
query2 = '''
SELECT name, COUNT(capital)
FROM world
GROUP BY name
HAVING COUNT(capital) > 1;
'''
result2 = pd.read_sql_query(query2,conn)
result2

Empty DataFrame
Columns: [name, COUNT(capital)]
Index: []

# Problem 3: Find countries and their respective populaltion and area with a population/area density greater than 1000. Show the largest density first.

In [ ]:
query3 = '''
SELECT name, population, area, population/area AS density_over_1000
FROM world
GROUP BY name
HAVING population/area >1000
ORDER BY population/area DESC
'''
result3 = pd.read_sql_query(query3,conn)
result3

name  population    area  density_over_1000
0      Monaco       36950       2              18475
1   Singapore     5399200     710               7604
2     Bahrain     1234571     765               1613
3       Malta      416055     316               1316
4  Bangladesh   156557000  147570               1060
5    Maldives      317280     300               1057

# Problem 4: List each country name where the population is larger than that of 'Japan'.

In [ ]:
query4 = '''
SELECT name, population
FROM world
WHERE population > 127090000
ORDER BY population DESC;
'''
result4 = pd.read_sql_query(query4,conn)
result4

name  population
0          China  1365370000
1          India  1246160000
2  United States   318320000
3      Indonesia   252164800
4         Brazil   202794000
5       Pakistan   188020000
6        Nigeria   178517000
7     Bangladesh   156557000
8         Russia   146000000

# Problem 5: Show the countries in Europe with a per capita GDP  (GDP/population) greater than 'United Kingdom'.

In [ ]:
query6 = '''
SELECT name, ROUND(GDP/population, 2) AS GDP_per_captia
FROM world
WHERE continent = 'Europe'
GROUP BY name
HAVING GDP/population > 38555.08
ORDER BY GDP/population DESC;
'''
result6 = pd.read_sql_query(query6,conn)
result6

name  GDP_per_captia
0   Liechtenstein       156926.64
1          Monaco       154451.96
2      Luxembourg       100314.72
3          Norway        97507.74
4     Switzerland        77342.33
5      San Marino        56776.05
6         Denmark        55886.51
7          Sweden        54135.00
8     Netherlands        47488.36
9         Austria        46380.36
10        Ireland        45859.66
11        Finland        45325.63
12        Belgium        43166.14
13        Germany        42444.57
14        Andorra        42340.14
15        Iceland        41609.98
16         France        39620.38

# Problem 6: What is the total population of ('Estonia', 'Latvia', 'Lithuania') combined!

In [ ]:
query6 = '''
SELECT SUM(population) AS total_population
FROM world
WHERE name IN ('Estonia', 'Latvia', 'Lithuania');
'''
result6 = pd.read_sql_query(query6,conn)
result6

total_population
0           6251750